In [1]:
!pip install pyspark faker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 41.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=694efec733681ae81737f8d0fde48afbcb871b516e080a7cb667895761b353d9
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
from faker import Faker
import random

spark = SparkSession.builder.appName('bank_balances').getOrCreate()
fake = Faker()

def generate_data(n):
  data = []
  for _ in range(n):
    name = fake.name()
    balance = round(random.uniform(1000, 10000), 2)
    account_number = fake.bban()
    data.append((name, balance, account_number))
  return data

dados = generate_data(1000)
colunas = ['name', 'balance', 'account_number']
df = spark.createDataFrame(dados, schema=colunas)
df.show(10)

+--------------------+-------+------------------+
|                name|balance|    account_number|
+--------------------+-------+------------------+
|          Gary Hayes| 6804.2|UZCN86335529906364|
|           Jason Cox|2339.95|JLSO52360330015912|
|       Nathan Obrien|2576.24|HMIG05886021373869|
|    Christine Haynes|6290.12|DIIX02620661775321|
|       Laura Trevino|7931.76|WRGZ35118705455099|
|Mr. Michael Powel...|8727.65|HATF61328720662941|
|        Dana Stewart|3743.91|XRED28900424704845|
|           Mark Dunn|7526.93|FGMO81836202319555|
|     Kristen Griffin|5026.93|SSGE89635513965905|
|           Tina Pace|6649.01|JCYR41743287013411|
+--------------------+-------+------------------+
only showing top 10 rows



In [3]:
df.count()

1000

In [5]:
# ordernação dos dados pelo saldo
df_ordenado = df.orderBy(df.balance.desc()).limit(10)
df_ordenado.show()

+----------------+-------+------------------+
|            name|balance|    account_number|
+----------------+-------+------------------+
|    Glen Daniels| 9996.7|LXRL15528638311326|
|     Jorge Avila|9990.84|TWBV34546599182158|
| Danielle Cooper|9983.05|QPTP31611227882970|
| Matthew Andrews|9974.73|DHYQ56717427049851|
|  Maureen Foster|9930.08|ZXDW08251516733153|
|  Thomas Leblanc|9919.38|QNWR73815850460081|
|   Thomas Martin|9918.24|ZHJA44191846895216|
| Shannon Morales|9905.56|BGFF45217296831708|
|Kendra Middleton| 9903.9|ZMGO61154906670523|
|   Raymond Greer|9885.78|BWXG85134468587749|
+----------------+-------+------------------+



In [7]:
# Busca de contas com saldo maior que um valor específico
valor = 5000
df_valor_acima = df.filter(df.balance > valor)
df_valor_acima.count()

575